In [ ]:
import numpy as np
import pandas as pd

from IPython.display import display

<h3>Drop tables</h3>

In [ ]:
%sql DROP TABLE IF EXISTS life_expectancy_2010_2015

In [ ]:
%sql DROP TABLE IF EXISTS life_expectancy_2018

In [ ]:
%sql DROP TABLE IF EXISTS us_regions

In [ ]:
%sql DROP TABLE IF EXISTS us_gdp

<h3>Create</h3>


In [ ]:
%sql CREATE TABLE life_expectancy_2010_2015 (
    State varchar,
    County varchar,
    CensusTractNumber varchar,
    LifeExpectancy decimal(4,2),
    LifeExpectancyRange varchar,
    LifeExpectancyStandardError decimal(4,2)
)

In [ ]:
%sql CREATE TABLE life_expectancy_2018 (
    State varchar,
    Sex varchar,
    LEB decimal(3,1),
    SE decimal(3,1),
    Quartile varchar
)

In [ ]:
%sql CREATE TABLE us_regions (
    State varchar,
    StateCode varchar(2),
    Region varchar,
    Division varchar
)

In [ ]:
%sql CREATE TABLE us_gdp (
    County varchar,
    Year2017 bigint,
    Year2018 bigint,
    Year2019 bigint,
    Year2020 bigint
)

<h3>Ingest Table Life expectancy 2010-2015</h3>

In [ ]:
def get_df():
    url = "http://data.cdc.gov/api/views/5h56-n989/rows.csv"
    dtypes = {
        "State": str,
        "County": str,
        "Census Tract Number": str,
        "Life Expectancy": np.float64,
        "Life Expectancy Range": str,
        "Life Expectancy Standard Error": np.float64,
    }
    df = pd.read_csv(url, dtype=dtypes).replace("'", "''", regex=True)
    return df

In [ ]:
def refactor_df(df):
    df.columns = df.columns.str.replace(" ", "")
    return df

In [ ]:
def refactor_df_in_other_way(df):
    #do some changes on df
    return df

In [ ]:
def run(job_input: IJobInput):
    df = refactor_df(get_df())
    job_input.send_tabular_data_for_ingestion(
        df.itertuples(index=False),
        destination_table="life_expectancy_2010_2015",
        column_names=df.columns.tolist(),
    )

<h5>Tests</h5>

In [ ]:
df = get_df()

In [ ]:
refactor_df(df)
display(df)

In [ ]:
refactor_df_in_other_way(df)
display(df)

<h3>Ingest Table Life expectancy 2018</h3>

In [ ]:

%run-start
 url = "http://data.cdc.gov/api/views/a5a8-jsrq/rows.csv"

    dtypes = {
        "State": str,
        "Sex": str,
        "LEB": np.float64,
        "SE": np.float64,
        "Quartile": str,
    }

In [ ]:
 df = pd.read_csv(url, dtype=dtypes, na_values="*")

In [ ]:
display(df)

In [ ]:
job_input.send_tabular_data_for_ingestion(
        df.itertuples(index=False),
        destination_table="life_expectancy_2018",
        column_names=df.columns.tolist(),
)
%run-end


<h3>Ingest Table US regions</h3>

In [ ]:
def get_df():
    url = "https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv"
    df = pd.read_csv(url, sep=",")
    return df

In [ ]:
def refactor_df(df):
    df.columns = df.columns.str.replace(" ", "")
    return df

In [ ]:
def run(job_input: IJobInput):
    df = refactor_df(get_df())
    job_input.send_tabular_data_for_ingestion(
        df.itertuples(index=False),
        destination_table="us_regions",
        column_names=df.columns.tolist(),
    )

<h3>Ingest Table US gdp</h3>

In [ ]:
%run-start
url = "https://www.bea.gov/sites/default/files/2021-12/lagdp1221.xlsx"
df = pd.read_excel(url, header=3, na_values="(NA)").replace("'", "''", regex=True)

In [ ]:
df = df[["Unnamed: 0", 2017, 2018, 2019, 2020]]

In [ ]:
 df.rename(
        {
            "Unnamed: 0": "County",
            2017: "Year2017",
            2018: "Year2018",
            2019: "Year2019",
            2020: "Year2020",
        },
        axis=1,
        inplace=True,
    )
    df.dropna(axis=0, inplace=True)

In [ ]:
 job_input.send_tabular_data_for_ingestion(
        df.itertuples(index=False),
        destination_table="us_gdp",
        column_names=df.columns.tolist(),
    )
%run-end

<h3>Delete tables</h3>

In [ ]:
%sql DROP TABLE IF EXISTS cleaned_life_expectancy_2010_2015

In [ ]:
%sql DROP TABLE IF EXISTS cleaned_life_expectancy_2018

In [ ]:
%sql DROP TABLE IF EXISTS merged_life_expectancy

<h3>Create and merge clean tables</h3>

In [ ]:
%sql CREATE TABLE cleaned_life_expectancy_2010_2015 AS
(SELECT State,
    LifeExpectancy,
    cast(split(life_expectancy_2010_2015.LifeExpectancyRange,'-')[1] AS decimal(4,2)) AS MinLifeExpectancyRange,
    cast(split(life_expectancy_2010_2015.LifeExpectancyRange,'-')[2] AS decimal(4,2)) AS MaxLifeExpectancyRange,
    LifeExpectancyStandardError
FROM life_expectancy_2010_2015
WHERE County = '(blank)'
)

In [ ]:
%sql CREATE TABLE cleaned_life_expectancy_2018 AS
(SELECT State,
    LEB AS LifeExpectancy,
    cast(split(life_expectancy_2018.Quartile,' - ')[1] AS decimal(4,2)) AS MinLifeExpectancyRange,
    cast(split(life_expectancy_2018.Quartile,' - ')[2] AS decimal(4,2)) AS MaxLifeExpectancyRange,
    SE AS LifeExpectancyStandardError
FROM life_expectancy_2018
WHERE Sex = 'Total' and State <> 'United States'
)

In [ ]:
%sql CREATE TABLE merged_life_expectancy AS
(SELECT us_regions.State,
    LifeExpectancy,
    MinLifeExpectancyRange,
    MaxLifeExpectancyRange,
    '2010-2015' AS Period,
    Region,
    0 AS GDP
FROM
    cleaned_life_expectancy_2010_2015 JOIN us_regions ON us_regions.State = cleaned_life_expectancy_2010_2015.State
)
UNION
(SELECT us_regions.State,
    LifeExpectancy,
    MinLifeExpectancyRange,
    MaxLifeExpectancyRange,
    '2018' AS Period,
    Region,
    Year2018 AS GDP
FROM cleaned_life_expectancy_2018
    JOIN us_regions ON us_regions.State = cleaned_life_expectancy_2018.State
    INNER JOIN us_gdp ON us_gdp.County = cleaned_life_expectancy_2018.State
WHERE Year2018 > 100000000
)